In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

In [2]:
# load the training and test data

train_data_path = "../data/processed/train.csv"
test_data_path = "../data/processed/test.csv"

train_df = pd.read_csv(train_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

test_df = pd.read_csv(test_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

train_df

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,194,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,180,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,197,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185,185.0,4
...,...,...,...,...,...,...,...,...
2016-02-29 22:45:00,15.0,9.0,11.0,11.0,29,12,12.0,0
2016-02-29 23:00:00,12.0,15.0,9.0,11.0,29,17,14.0,0
2016-02-29 23:15:00,17.0,12.0,15.0,9.0,29,15,14.0,0


In [3]:
# missing value in training data

train_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [4]:
# missing values in the test data

test_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [5]:
# make X_train and y_train

X_train = train_df.drop(columns=["total_pickups"])

y_train = train_df["total_pickups"]

In [6]:
X_train.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185.0,4


In [7]:
# make X_test and y_test

X_test = test_df.drop(columns=["total_pickups"])

y_test = test_df["total_pickups"]

In [8]:
X_test.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-03-01 00:00:00,36.0,44.0,31.0,29.0,0,39.0,1
2016-03-01 00:15:00,41.0,36.0,44.0,31.0,0,37.0,1
2016-03-01 00:30:00,35.0,41.0,36.0,44.0,0,41.0,1
2016-03-01 00:45:00,47.0,35.0,41.0,36.0,0,38.0,1
2016-03-01 01:00:00,34.0,47.0,35.0,41.0,0,35.0,1


In [9]:
from sklearn import set_config

set_config(transform_output="pandas")

In [10]:
# encode the data

encoder = ColumnTransformer([
    ("ohe", OneHotEncoder(drop="first",sparse_output=False), ["region","day_of_week"])
], remainder="passthrough", n_jobs=-1,force_int_remainder_cols=False)

In [11]:
encoder

,transformers,"[('ohe', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,False
,categories,'auto'
,drop,'first'
,sparse_output,False


In [12]:
# encode the train and test data

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

c:\Users\akshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


In [13]:
import optuna
import tqdm 


In [14]:
def objective(trial):

    # model name search space
    list_of_models = ["LR", "RIDGE",  "RF", "GBR", "XGBR"]
    model_name = trial.suggest_categorical("model_name", list_of_models)

    if model_name == "LR":
        model = LinearRegression()

    elif model_name == "RIDGE":
        ridge_alpha = trial.suggest_float("ridge_alpha",1,50)
        model = Ridge(alpha=ridge_alpha)

    elif model_name == "RF":
        n_estimators_rf = trial.suggest_int("n_estimators_rf",10,100,step=10)
        max_depth_rf = trial.suggest_int("max_depth_rf",3,10)
        model = RandomForestRegressor(n_estimators=n_estimators_rf, 
                                      max_depth=max_depth_rf, 
                                      random_state=42, n_jobs=-1)

    elif model_name == "GBR":
        n_estimators_gb = trial.suggest_int("n_estimators_gb",10,100,step=10)
        learning_rate_gb = trial.suggest_float("learning_rate_gb",1e-4,1e-1)
        model = GradientBoostingRegressor(n_estimators=n_estimators_gb, 
                                          learning_rate=learning_rate_gb,
                                         random_state=42)

    elif model_name == "XGBR":
        n_estimators_xgb = trial.suggest_int("n_estimators_xgb",10,100,step=10)
        learning_rate_xgb = trial.suggest_float("learning_rate_xgb",1e-4,1e-1)
        max_depth_xgb = trial.suggest_int("max_depth_xgb",3,10)
        model = XGBRegressor(n_estimators=n_estimators_xgb,
                            learning_rate=learning_rate_xgb,
                            max_depth=max_depth_xgb)

    # fit on the data
    model.fit(X_train_encoded,y_train)

    # get the predictions
    y_pred = model.predict(X_test_encoded)

    # calculate the loss
    loss = mean_absolute_percentage_error(y_test, y_pred)

    return loss

In [15]:
# create a study object
study = optuna.create_study(study_name="model_selection", direction="minimize")

[I 2025-12-24 17:04:29,918] A new study created in memory with name: model_selection


In [16]:
# optimize the objective function
study.optimize(func=objective, n_trials=50, n_jobs=-1)

[I 2025-12-24 17:04:30,795] Trial 11 finished with value: 0.07926455347525414 and parameters: {'model_name': 'RIDGE', 'ridge_alpha': 35.535670366534774}. Best is trial 11 with value: 0.07926455347525414.
[I 2025-12-24 17:04:30,795] Trial 6 finished with value: 0.07934790285463077 and parameters: {'model_name': 'LR'}. Best is trial 11 with value: 0.07926455347525414.
[I 2025-12-24 17:04:30,806] Trial 9 finished with value: 0.07923878035436031 and parameters: {'model_name': 'RIDGE', 'ridge_alpha': 49.30158357917938}. Best is trial 9 with value: 0.07923878035436031.
[I 2025-12-24 17:04:30,829] Trial 5 finished with value: 0.07924391075905912 and parameters: {'model_name': 'RIDGE', 'ridge_alpha': 46.41662730489897}. Best is trial 9 with value: 0.07923878035436031.
[I 2025-12-24 17:04:30,870] Trial 7 finished with value: 0.07934790285463077 and parameters: {'model_name': 'LR'}. Best is trial 9 with value: 0.07923878035436031.
[I 2025-12-24 17:04:34,135] Trial 16 finished with value: 0.07934

In [17]:
# best value

study.best_value

0.07923762414475963

In [18]:
# best parameters

study.best_params

{'model_name': 'RIDGE', 'ridge_alpha': 49.96136201785073}

In [19]:
# model value counts

study.trials_dataframe()['params_model_name'].value_counts()

params_model_name
XGBR     15
RIDGE    14
RF        9
GBR       7
LR        5
Name: count, dtype: int64

In [20]:
from optuna.visualization import (
    plot_optimization_history, 
    plot_parallel_coordinate, 
    plot_param_importances
)

In [21]:
import sys
print(sys.executable)

import plotly
print(plotly.__version__)

c:\Users\akshu\AppData\Local\Programs\Python\Python312\python.exe
6.5.0


In [22]:
plot_optimization_history(study)

In [23]:
plot_parallel_coordinate(study, params=["model_name"])

In [24]:
plot_parallel_coordinate(study, params=["ridge_alpha"])

In [25]:
# train the linear regression model

lr = LinearRegression()

lr.fit(X_train_encoded, y_train)

# get predictions
y_pred_train = lr.predict(X_train_encoded) 
y_pred_test = lr.predict(X_test_encoded)

# loss

mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

print("The training error is ", mape_train)
print("The test error is ", mape_test)

The training error is  0.0877801330456651
The test error is  0.07934790285463077


In [26]:
# train the ridge regression model

ridge = Ridge(alpha=50, random_state=42)

ridge.fit(X_train_encoded, y_train)

# get predictions
y_pred_train = ridge.predict(X_train_encoded) 
y_pred_test = ridge.predict(X_test_encoded)

# loss

mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

print("The training error is ", mape_train)
print("The test error is ", mape_test)

The training error is  0.08755473541886993
The test error is  0.07923755658782361


In [27]:
def tune_ridge(trial):
    # hyperparameter space
    alpha = trial.suggest_float("alpha",30,100)
    
    # make the model object
    ridge = Ridge(alpha=alpha, random_state=42)
    
    # train the model
    ridge.fit(X_train_encoded, y_train)
    
    # get predictions
    y_pred = ridge.predict(X_test_encoded)
    
    # calculate loss
    loss = mean_absolute_percentage_error(y_test, y_pred)

    return loss
        

In [28]:
# create study

study = optuna.create_study(study_name="tune_model", direction="minimize")

[I 2025-12-24 17:05:22,823] A new study created in memory with name: tune_model


In [29]:
# optimize

study.optimize(func=tune_ridge, n_trials=100, n_jobs=-1, show_progress_bar=True)

  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-12-24 17:05:23,275] Trial 0 finished with value: 0.0791994161800493 and parameters: {'alpha': 73.95922451242458}. Best is trial 0 with value: 0.0791994161800493.
[I 2025-12-24 17:05:23,275] Trial 2 finished with value: 0.07916564705536153 and parameters: {'alpha': 98.71498843456467}. Best is trial 2 with value: 0.07916564705536153.
[I 2025-12-24 17:05:23,275] Trial 6 finished with value: 0.07923604970653782 and parameters: {'alpha': 50.86688398467666}. Best is trial 2 with value: 0.07916564705536153.
[I 2025-12-24 17:05:23,275] Trial 3 finished with value: 0.07917220112990192 and parameters: {'alpha': 93.53378393169231}. Best is trial 2 with value: 0.07916564705536153.
[I 2025-12-24 17:05:23,275] Trial 7 finished with value: 0.07924665722341308 and parameters: {'alpha': 44.90382344621279}. Best is trial 2 with value: 0.07916564705536153.
[I 2025-12-24 17:05:23,291] Trial 4 finished with value: 0.0792632918806794 and parameters: {'alpha': 36.175193644719315}. Best is trial 2 wit

In [30]:
# best parameters

study.best_params

{'alpha': 99.99332952968061}

In [31]:
# best value

study.best_value

0.07916407374883087

In [32]:
# train the ridge regression model

ridge = Ridge(alpha=100, random_state=42)

ridge.fit(X_train_encoded, y_train)

# get predictions
y_pred_train = ridge.predict(X_train_encoded) 
y_pred_test = ridge.predict(X_test_encoded)

# loss

mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

print("The training error is ", mape_train)
print("The test error is ", mape_test)

The training error is  0.08739324289543417
The test error is  0.07916406556067876
